# Advanced Python - Building Scalable Applications

### Module 7

#### Network socket programming
 -  Using ```asyncio``` and ```trio``` for concurrent socket programming

#### Web (RESTful) API using Python
 - An overview on RESTful API architecture
 - Consuming web API using python requests and httpx
 - An overview on WSGI and ASGI
 - WSGI / ASGI servers: ```gunicorn``` and ```uvicorn```
 - An overview on ```starlette``` ASGI framework
 - Creating and exposing web API using ```FastAPI``` framework
 - Discovering web APIs using ```/docs``` (Open-API) and ```/redocs``` interface
 - Creating validating models using ```pydantic```
 - Security Considerations: Authentication and Authorization techniques
